# Simple Lya P1D forecasts using minimizer and MCMC

In [1]:
%matplotlib inline
import os
import numpy as np
from cobaya.yaml import yaml_load_file
from cobaya.run import run

Load pre-run chains for Lya BAO and gal BAO

In [2]:
info = yaml_load_file('../inifiles/test.yaml')
info['force']=True
info['debug']=False
info['output']=None
_, sampler = run(info)

[camb] Importing *auto-installed* CAMB (but defaulting to *global*).
[camb] Initialized!
[minimize] Initializing
[minimize] Starting from random initial point:
[minimize] {'ns': 0.9578880180602191, 'logA': 2.9966079522371523}
[minimize] Starting minimization.
[minimize] Finished successfully!
[minimize] -log(posterior) minimized to -0.77651
[minimize] Parameter values at minimum:
   weight  minuslogpost        ns     logA            As     ombh2     omch2  minuslogprior  minuslogprior__0      chi2  chi2__lyalike.lya.Tester
0     1.0      -0.77651  0.964462  3.02958  2.068854e-09  0.021996  0.120109      -0.776529         -0.776529  0.000037                  0.000037


In [3]:
def get_covariance(sampler):
    p = sampler.products()
    r = p['result_object']
    try:
        # method: scipy
        inv_hess = r.hess_inv.todense()
    except:
        # method: bobyqa
        inv_hess = np.linalg.inv(r.hessian)
    cov=p['M'].T.dot(inv_hess).dot(p['M'])
    return cov

In [4]:
cov=get_covariance(sampler)
min_err=np.sqrt(cov.diagonal())
min_ns=sampler.products()['minimum']['ns']
min_logA=sampler.products()['minimum']['logA']
print('ns = {:.5e} +/- {:.5e}'.format(min_ns,min_err[0]))
print('logA = {:.5e} +/- {:.5e}'.format(min_logA,min_err[1]))

ns = 9.64462e-01 +/- 2.68436e-03
logA = 3.02958e+00 +/- 9.85926e-03


### Redo with MCMC chains

In [5]:
info_mcmc = yaml_load_file('../inifiles/test.yaml')
info_mcmc['sampler']={'mcmc':{'max_tries':1000}}
info_mcmc['force']=True
info_mcmc['debug']=False
info_mcmc['output']=None
updated_info_mcmc, sampler_mcmc = run(info_mcmc)

[camb] Importing *auto-installed* CAMB (but defaulting to *global*).
[camb] Initialized!
[mcmc] Getting initial point... (this may take a few seconds)
[model] Measuring speeds... (this may take a few seconds)
[model] Setting measured speeds (per sec): {lyalike.lya.Tester: 34500.0, camb.transfers: 1.43, camb: 1600.0, lyalike.lya.Lya: 1610.0}
[mcmc] Covariance matrix not present. We will start learning the covariance of the proposal earlier: R-1 = 30 (would be 2 if all params loaded).
[mcmc] Initial point: ns:0.9605521, logA:2.889106
[mcmc] Sampling!
[mcmc] Progress @ 2021-03-15 05:35:08 : 1 steps taken, and 0 accepted.
[mcmc] Learn + convergence test @ 80 samples accepted.
[mcmc]  - Acceptance rate: 0.031
[mcmc]  - Convergence of means: R-1 = 0.624957 after 80 accepted steps
[mcmc]  - Updated covariance matrix of proposal pdf.
[mcmc] Learn + convergence test @ 160 samples accepted.
[mcmc]  - Acceptance rate: 0.059
[mcmc]  - Convergence of means: R-1 = 0.015186 after 160 accepted steps
[

In [6]:
# Export the results to GetDist
from getdist.mcsamples import MCSamplesFromCobaya

gd_sample = MCSamplesFromCobaya(updated_info_mcmc, sampler_mcmc.products()["sample"])
# Analyze and plot
mcmc_mean = gd_sample.getMeans()[:2]
mcmc_cov = gd_sample.getCovMat().matrix[:2, :2]
mcmc_err = np.sqrt(mcmc_cov.diagonal())
print('mcmc ns = {:.5e} +/- {:.5e}'.format(mcmc_mean[0],mcmc_err[0]))
print('mcmc logA = {:.5e} +/- {:.5e}'.format(mcmc_mean[1],mcmc_err[1]))

[root] *WARNING* outlier fraction 0.05078125 
mcmc ns = 9.64688e-01 +/- 2.87093e-03
mcmc logA = 3.02946e+00 +/- 9.31773e-03


In [7]:
print('minimizer ns = {:.5e} +/- {:.5e}'.format(min_ns,min_err[0]))
print('minimizer logA = {:.5e} +/- {:.5e}'.format(min_logA,min_err[1]))

minimizer ns = 9.64462e-01 +/- 2.68436e-03
minimizer logA = 3.02958e+00 +/- 9.85926e-03
